In [3]:
! pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 42.1 MB/s eta 0:00:00


In [75]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.circuit.library import MCXGate
from qiskit.visualization import plot_histogram , plot_bloch_multivector
import numpy as np

In [113]:
def oracle(n,x):
  qc = QuantumCircuit(n)
  s = []

  for ind , i in enumerate(x):  # x is from least to mostsignificant
    if i == '0':
      s.append(ind)
      qc.x(ind)

  # multicontrolled controlled z
  qc.h(n-1)
  qc.mcx(list(range(n-1)),n-1)
  qc.h(n-1)

  if s:
    for i in s:
        qc.x(i)

  return qc

def diffusion(n):
  qc = QuantumCircuit(n,n)

  qc.h(range(n))
  qc.x(range(n))

  # multicontrolled z gate
  qc.h(n-1)
  qc.mcx(list(range(n-1)),n-1)
  qc.h(n-1)

  qc.x(range(n))
  qc.h(range(n))

  return qc

In [107]:
def grovers_algo(n,x,iterations=1,m=1):
  if iterations == 'auto':
    iterations = int(np.floor((np.pi/4) * np.sqrt(2**n/m) ))

  qc = QuantumCircuit(n,n)
  qc.h(range(n))

  # two operations of Grover's operation
  for _ in range(iterations):
    print("iteration")
    qc.barrier(label='iter start')

    # ist  Uf (sign flip) assuming 2 bit input and x' = 11
    qc = qc.compose(oracle(n,x))
    qc.barrier()

    # 2nd W (Diffusion Operation)
    qc = qc.compose(diffusion(n))
    qc.barrier(label='iter end')

  qc.measure(range(n),range(n))

  return qc

In [114]:
qc = grovers_algo(5,x='10111',iterations='auto')

iteration
iteration
iteration
iteration


In [115]:
qc.draw()

┌───┐ iter start                 ░ ┌───┐┌───┐          ┌───┐┌───┐     »
q_0: ┤ H ├─────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────»
     ├───┤     ░      ┌───┐  │  ┌───┐ ░ ├───┤├───┤       │  ├───┤├───┤     »
q_1: ┤ H ├─────░──────┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────»
     ├───┤     ░      └───┘  │  └───┘ ░ ├───┤├───┤       │  ├───┤├───┤     »
q_2: ┤ H ├─────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────»
     ├───┤     ░             │        ░ ├───┤├───┤       │  ├───┤├───┤     »
q_3: ┤ H ├─────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────»
     ├───┤     ░      ┌───┐┌─┴─┐┌───┐ ░ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐»
q_4: ┤ H ├─────░──────┤ H ├┤ X ├┤ H ├─░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├»
     └───┘     ░      └───┘└───┘└───┘ ░ └───┘└───┘└───┘└───┘└───┘└───┘└───┘»
c: 5/══════════════════════════════════════════════════════════════════════»
                                                                           »
«      iter end  iter start                 ░ ┌───┐┌───┐          ┌───┐┌───┐»
«q_0: ────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├»
«         ░          ░      ┌───┐  │  ┌───┐ ░ ├───┤├───┤       │  ├───┤├───┤»
«q_1: ────░──────────░──────┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├»
«         ░          ░      └───┘  │  └───┘ ░ ├───┤├───┤       │  ├───┤├───┤»
«q_2: ────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├»
«         ░          ░             │        ░ ├───┤├───┤       │  ├───┤├───┤»
«q_3: ────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├┤ H ├»
«         ░          ░      ┌───┐┌─┴─┐┌───┐ ░ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤»
«q_4: ────░──────────░──────┤ H ├┤ X ├┤ H ├─░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├»
«         ░          ░      └───┘└───┘└───┘ ░ └───┘└───┘└───┘└───┘└───┘└───┘»
«c: 5/══════════════════════════════════════════════════════════════════════»
«                                                                           »
«           iter end  iter start                 ░ ┌───┐┌───┐          ┌───┐»
«q_0: ─────────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├»
«              ░          ░      ┌───┐  │  ┌───┐ ░ ├───┤├───┤       │  ├───┤»
«q_1: ─────────░──────────░──────┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├───────■──┤ X ├»
«              ░          ░      └───┘  │  └───┘ ░ ├───┤├───┤       │  ├───┤»
«q_2: ─────────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├»
«              ░          ░             │        ░ ├───┤├───┤       │  ├───┤»
«q_3: ─────────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──┤ X ├»
«     ┌───┐    ░          ░      ┌───┐┌─┴─┐┌───┐ ░ ├───┤├───┤┌───┐┌─┴─┐├───┤»
«q_4: ┤ H ├────░──────────░──────┤ H ├┤ X ├┤ H ├─░─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├»
«     └───┘    ░          ░      └───┘└───┘└───┘ ░ └───┘└───┘└───┘└───┘└───┘»
«c: 5/══════════════════════════════════════════════════════════════════════»
«                                                                           »
«     ┌───┐      iter end  iter start                 ░ ┌───┐┌───┐          »
«q_0: ┤ H ├─────────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──»
«     ├───┤         ░          ░      ┌───┐  │  ┌───┐ ░ ├───┤├───┤       │  »
«q_1: ┤ H ├─────────░──────────░──────┤ X ├──■──┤ X ├─░─┤ H ├┤ X ├───────■──»
«     ├───┤         ░          ░      └───┘  │  └───┘ ░ ├───┤├───┤       │  »
«q_2: ┤ H ├─────────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──»
«     ├───┤         ░          ░             │        ░ ├───┤├───┤       │  »
«q_3: ┤ H ├─────────░──────────░─────────────■────────░─┤ H ├┤ X ├───────■──»
«     ├───┤┌───┐    ░          ░      ┌───┐┌─┴─┐┌───┐ ░ ├───┤├───┤┌───┐┌─┴─┐»
«q_4: ┤ X ├┤ H ├────░──────────░──────┤ H ├┤ X ├┤ H ├─░─┤ H ├┤ X ├┤ H ├┤ X ├»
«     └───┘└───┘    ░          ░      └───┘└───┘└───┘ ░ └───┘└───┘└───┘└───┘»
«c: 5/══════════════════════════════════════════════════════════════════════»
«                                       

In [1]:
sim = Aer.get_backend('aer_simulator')
result = sim.run(qc).result()
plot_histogram(result.get_counts()) # x is most significant to least significant

NameError: name 'Aer' is not defined